In [1]:
import torch
from Scaled_KAN import FastKAN
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_conv = FastKAN(
                layers_hidden=[784,1569,10],
                num_grids=4,
                device=device
            )
model_continual = FastKAN(
                layers_hidden=[784,128,10],
                num_grids=4,
                device=device
            )

model_conv.load_state_dict(torch.load('FastKAN_4_6M.pth'))
model_conv.eval()

model_continual.load_state_dict(torch.load('continual_mnist_FastKAN_128.pth'))
model_continual.eval()

layers_conv = []
layers_continual = []
for layer in model_conv.modules():
    layers_conv.append(layer)
for layer in model_continual.modules():
    layers_continual.append(layer)

layer_conv = layers_conv[1]
layer_continual = layers_continual[1]



# Assuming layer[1].plot_curve is a function that takes two arguments and returns x, y coordinates
def plot_curve_with_slider(j):
    x_conv, y_conv = layer_conv[1].plot_curve(0, j, num_pts=100, num_extrapolate_bins=3)
    x_continual, y_continual = layer_continual[1].plot_curve(0, j, num_pts=100, num_extrapolate_bins=3)
    plt.figure()
    plt.plot(x_conv.to('cpu').numpy(), y_conv.to('cpu').numpy(), label=f"Last Layer, Input 0, Output {j}")
    plt.plot(x_continual.to('cpu').numpy(), y_continual.to('cpu').numpy(), label=f"Last Layer, Input 0, Output {j}")
    plt.legend()
    plt.show()

# Create an interactive slider
interactive_plot = interactive(plot_curve_with_slider, j=widgets.SelectionSlider(options=[i for i in range(10)]))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(SelectionSlider(description='j', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Outpu…